## Playing Pong Using Only Pixel Values ft. Policy Gradient

In [1]:
# Import stuff
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time, os
from torch.utils.tensorboard import SummaryWriter

# Set random seed
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Create directory for tensorlog
# Make sure to use a new directory for every new run
log_dir = 'logs/pong_pg_test'
writer = SummaryWriter(log_dir)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# Create the environment
env = gym.make("Pong-v0")
env.seed(seed)

n_actions = env.action_space.n
print(n_actions)

6


In [3]:
# A function to grab the current game screen and return it as a 2D numpy array
def get_image(env):
    image = env.render(mode='rgb_array')
    image = image.astype(np.float32) / 255.0  # convert to float and scale to the range [0,1]
    image = np.dot(image, [0.299, 0.587, 0.114])  # convert to grayscale
    return image

In [4]:
# A class for the policy network. Here, we will use a convolutional neural network
# that will take an entire screen of game state and suggest an action from that.
class PolicyNetwork(nn.Module):
    def __init__(self, input_channels=1, input_height=210, input_width=160, output_size=6):
        """Initialize the network"""
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=16, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2)
        # Can try using pooling layers here
        flat_size = 32 * self.conv2d_size_out(self.conv2d_size_out(input_height, 8, 4), 4, 2) * \
                    self.conv2d_size_out(self.conv2d_size_out(input_width, 8, 4), 4, 2)
        self.fc1 = nn.Linear(flat_size, 256)
        self.fc2 = nn.Linear(256, output_size)
        
    def conv2d_size_out(self, size, kernel_size, stride):
        """Utility function to calculate size of dimension after convolution"""
        return (size - (kernel_size - 1) - 1) // stride + 1
        
    def forward(self, x):
        """Make a forward pass"""
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

In [5]:
# A class for the agent
class Agent:
    def __init__(self, learning_rate=0.001, n_actions=6):
        """Initialize agent"""
        self.learning_Rate = learning_rate
        self.n_actions = n_actions
        self.policy = PolicyNetwork().to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).unsqueeze(0)
        state = state.to(device)
        probs = self.policy(state)
        action = np.random.choice(self.n_actions, p=probs.to('cpu').detach().squeeze(0).numpy())
        log_prob = torch.log(probs.squeeze(0)[action])
        state = state.to('cpu')
        return action, log_prob
    
    def get_probs(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).unsqueeze(0)
        state = state.to(device)
        probs = self.policy(state)
        state = state.to('cpu')
        return probs
    
    def update(self, rewards, log_probs, end_flags, discount_rate=0.9):
        # Compute discounted rewards
        discounted_rewards = [0] * len(rewards)
        total_reward = 0
        for i in reversed(range(len(rewards))):
            if end_flags[i] is True:
                total_reward = 0
            discounted_rewards[i] = rewards[i] + discount_rate * total_reward
            total_reward = discounted_rewards[i]
        
        # Convert to tensors
        discounted_rewards = torch.tensor(discounted_rewards).to(device)
        log_probs = torch.stack(log_probs).to(device)
        
        # Normalize rewards, this should speed up training
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        # Reset parameter gradients
        self.optimizer.zero_grad()
        
        # Compute "loss" function
        loss = torch.mul(discounted_rewards, -log_probs).mean()
        
        # Perform backprop
        loss.backward()
        
        # Optimize
        self.optimizer.step()
        
    def save_parameters(self, path):
        torch.save(self.policy.state_dict(), path)
        
    def load_parameters(self, path):
        self.policy.load_state_dict(torch.load(path))

In [6]:
# A function to simulate an episode
def simulate(env, agent, render=False, fps=30, max_steps=10000, detailed=False):
    seconds_per_frame = 1 / fps
    total_reward = 0
    env.reset()
    image = get_image(env)
    prev_image = image
    state = image - prev_image
    
    for i in range(max_steps):
        if render:
            env.render()
            time.sleep(seconds_per_frame)
            
        if detailed:
            probs = agent.get_probs(state)
            print(probs)
            
        action, log_prob = agent.get_action(state)
        _, reward, done, _ = env.step(action)
        total_reward += reward
        
        if done:
            env.close()
            break
        else:
            prev_image = image
            image = get_image(env)
            state = image - prev_image
    
    if detailed:
        print("Simulation complete - total reward:", total_reward)
    
    return total_reward

In [7]:
# Agent hyperparameters
learning_rate = 0.0001

# Initialize the agent
agent = Agent(learning_rate)

# Autosave settings
save_parameters = True
save_interval = 10
save_path = 'models/pg_pong_test.pth'
load_parameters_before_training = True

if load_parameters_before_training and os.path.exists(save_path):
    agent.load_parameters(save_path)
    print("Parameters loaded successfully")

Parameters loaded successfully


In [8]:
# Training hyperparameters
n_epochs = 10000
max_steps = 5000
discount_rate = 0.99

# Option to show the agent in training
show_simulation = False
epoch_per_simulation = 50

# Initialize environment
env.reset()
image = get_image(env)
prev_image = image
state = image - prev_image

# Training loop
for epoch in range(n_epochs):
    rewards = []
    log_probs = []
    end_flags = []
    
    for step in range(max_steps):
        action, log_prob = agent.get_action(state)
        _, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        end_flags.append(done)

        if done:
            # This episode is finished, start a new one
            env.reset()
            image = get_image(env)
            prev_image = image
            state = image - prev_image
        else:
            prev_image = image
            image = get_image(env)
            state = image - prev_image
        
    # Update the policy
    agent.update(rewards, log_probs, end_flags, discount_rate)
    
    # Track average rewards in TensorBoard
    writer.add_scalar('simulation_reward', simulate(env, agent), epoch)
    
    # Save model parameters
    if save_parameters:
        if epoch % save_interval == 0:
            agent.save_parameters(save_path)
    
    # Simulate agent (optional)
    if show_simulation and epoch % epoch_per_simulation == 0:
        simulate(env, agent, render=True)

KeyboardInterrupt: 

In [9]:
simulate(env, agent, render=True, detailed=False)

-9.0

In [10]:
env.close()
writer.close()